In [8]:
import os
import pandas as pd
import pickle
import google.oauth2.credentials

import googleapiclient.discovery

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

# The CLIENT_SECRETS_FILE variable specifies the name of a file that contains
# the OAuth 2.0 information for this application, including its client_id and
# client_secret.
CLIENT_SECRETS_FILE = "client_secret.json"

# This OAuth 2.0 access scope allows for full read/write access to the
# authenticated user's account and requires requests to use an SSL connection.
SCOPES = ['https://www.googleapis.com/auth/youtube.force-ssl']
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'


def get_authenticated_service():
    credentials = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            credentials = pickle.load(token)
    #  Check if the credentials are invalid or do not exist
    if not credentials or not credentials.valid:
        # Check if the credentials have expired
        if credentials and credentials.expired and credentials.refresh_token:
            credentials.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                CLIENT_SECRETS_FILE, SCOPES)
            credentials = flow.run_console()

        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(credentials, token)

    return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)

In [4]:
def main(url):
    # Disable OAuthlib's HTTPS verification when running locally.
    # *DO NOT* leave this option enabled in production.
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

    api_service_name = "youtube"
    api_version = "v3"
    DEVELOPER_KEY = "YOUR_API_KEY"
    service = get_authenticated_service()
    
# Get Video ID from URL
    splitURL=url.split("v=")
    videoId=splitURL[-1]

#Get video title
    requestVideo = service.videos().list(
        part="snippet,contentDetails,statistics",
        id=videoId
    )
    responseVideo = requestVideo.execute()
    
    for item in responseVideo['items']:
        videoTitle=item['snippet']['title']
        
# Get comments for the required video
    requestComment = service.commentThreads().list(
        part="snippet",
        videoId=videoId
    )
    responseComment = requestComment.execute()
    
    comment = []
    comment_id = []
    author=[]
    reply_count = []
    like_count = []
    i=0
     
    for item in responseComment['items']:
        comment.append(item['snippet']['topLevelComment']['snippet']['textDisplay'])
        comment_id.append(item['snippet']['topLevelComment']['id'])
        author.append(item['snippet']['topLevelComment']['snippet']['authorDisplayName'])
        reply_count.append(item['snippet']['totalReplyCount'])
        like_count.append(item['snippet']['topLevelComment']['snippet']['likeCount'])

    i = 0
    max_pages = 100
    final_results=[]
    while responseComment and i < max_pages:
        i=i+1
        print(i)
       
        final_results.extend(responseComment['items'])

        # Check if another page exists
        if 'nextPageToken' in final_results:
            print('next token')
            requestComment = service.commentThreads().list(
                part="snippet",
                videoId=videoId,
                pageToken=responseComment['nextPageToken'])
            responseComment = requestComment.execute()

            comment.append(item['snippet']['topLevelComment']['snippet']['textDisplay'])
            comment_id.append(item['snippet']['topLevelComment']['id'])
            author.append(item['snippet']['topLevelComment']['snippet']['authorDisplayName'])
            reply_count.append(item['snippet']['totalReplyCount'])
            like_count.append(item['snippet']['topLevelComment']['snippet']['likeCount'])

    else:  
        print('Inside else')
        doc=pd.DataFrame({ 
                      'Video Title':videoTitle,
                      'Comment': comment,
                      'Comment ID': comment_id,
                      'Author':author,
                      'Replies': reply_count,
                      'Likes': like_count})
      
    return(doc)      

        
if __name__ == "__main__":
    url = input('Enter an URL: ')
    doc=main(url)
#     print(doc.head(50))
    

Enter an URL: https://www.youtube.com/watch?v=QRhGxFl0NI4
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
Inside else


In [113]:
doc.drop_duplicates()

,Video Title,Comment,Comment ID,Author,Replies,Likes
0,Reading minds through body language | Lynne Fr...,<b>Lookers are Hooker&#39;s</b> 😂,UgwcbMrGRc3kWJBf1iR4AaABAg,Fisabilillah فی سبیل الله,0,0
1,Reading minds through body language | Lynne Fr...,I don&#39;t know if i buy this. Too simplistic.,UgyPO8VX5UdWk6DUGOd4AaABAg,badgothnobiscuit,0,0
2,Reading minds through body language | Lynne Fr...,James is her son.,UgwXgs2iMrecxONVqS54AaABAg,J Mac,0,0
3,Reading minds through body language | Lynne Fr...,These all seem like overgeneralizing 👎🏿,UgyvVXLHcwlV-8cqbtx4AaABAg,Angelo Pressley,0,0
4,Reading minds through body language | Lynne Fr...,Cool,UgwbDbsq9tgGlZHYY1J4AaABAg,Barbell King,0,0
5,Reading minds through body language | Lynne Fr...,I&#39;m a mix of looker &amp; listener i guess...,UgwbogSsnPRiNfaAHzR4AaABAg,TentruTented,0,0
6,Reading minds through body language | Lynne Fr...,Daffy deductionist twaddle.,Ugw67pDDnjFinBWidJR4AaABAg,Netizen Capet,0,0
7,Reading minds through body language | Lynne Fr...,you got a domiant style,Ugyq8cTL-vglgSvuxOh4AaABAg,Charles Webb,0,0
8,Reading minds through body language | Lynne Fr...,You made path of my crazy interest,Ugz-R4jJeSw2OGJm86J4AaABAg,Uday shankar Misra,0,1
9,Reading minds through body language | Lynne Fr...,Thx to my mom i learned to know a person insta...,UgxTs9riwGKSVIInF5h4AaABAg,Nobody,2,0


In [6]:
doc.shape

(20, 6)

##### EDA

In [74]:
service = get_authenticated_service()
requestComment = service.commentThreads().list(
        part="snippet",
        videoId='a0hL0-JMh2Q'
    )
responseComment = requestComment.execute()

In [91]:
responseComment['items'][0]['snippet']['topLevelComment']['snippet']['textDisplay']

'hello can you prepare idli and dosai and speaking more quiet thank from france'

In [94]:
responseComment['items']

[{'kind': 'youtube#commentThread',
  'etag': '"Fznwjl6JEQdo1MGvHOGaz_YanRU/rkkpJ6odwNYR-CoMoRRqq3WZ-Xw"',
  'id': 'Ugz2F4qmxxvw19JfI3p4AaABAg',
  'snippet': {'videoId': 'a0hL0-JMh2Q',
   'topLevelComment': {'kind': 'youtube#comment',
    'etag': '"Fznwjl6JEQdo1MGvHOGaz_YanRU/og4FpdpVLL62SZie41OY-g5hnbM"',
    'id': 'Ugz2F4qmxxvw19JfI3p4AaABAg',
    'snippet': {'authorDisplayName': 'gil sisu sisu',
     'authorProfileImageUrl': 'https://yt3.ggpht.com/a/AGF-l7_VtvrNLMUuwKx9a9HQo7_ZA0Bo3DLYMuZ6Ug=s48-c-k-c0xffffffff-no-rj-mo',
     'authorChannelUrl': 'http://www.youtube.com/channel/UC0EEq2SUYGztBR2U9AhhjHw',
     'authorChannelId': {'value': 'UC0EEq2SUYGztBR2U9AhhjHw'},
     'videoId': 'a0hL0-JMh2Q',
     'textDisplay': 'hello can you prepare idli and dosai and speaking more quiet thank from france',
     'textOriginal': 'hello can you prepare idli and dosai and speaking more quiet thank from france',
     'canRate': True,
     'viewerRating': 'none',
     'likeCount': 0,
     'published

In [62]:
service = get_authenticated_service()
requestVideo = service.videos().list(
        part="snippet,statistics",
        id="a0hL0-JMh2Q"
    )
responseVideo = requestVideo.execute()

In [73]:
responseComment.keys()

dict_keys(['kind', 'etag', 'nextPageToken', 'pageInfo', 'items'])

In [16]:
responseComment['nextPageToken']

'QURTSl9pMTlQcldrVC1HbUhsSVFtVFBBdkNYQVRKNThwd3FOSk1mdXRlb0dtYkNvaVZwNklKZW9lR1pPclBRVm1yT2tZaTJuTlVhQVVFUzlSc0x4TkFCNWk2UGFDejR3dEhwZ21iUkQ5d0ZCN1gtRXB3MS1UWjJOM2dFQXByRGxQR28='

In [41]:
for i in responseVideo['items']:
    print(i)

{'kind': 'youtube#video', 'etag': '"Fznwjl6JEQdo1MGvHOGaz_YanRU/7daS3g69kEQd8akXNrdevENW8c4"', 'id': 'a0hL0-JMh2Q', 'statistics': {'viewCount': '214738', 'likeCount': '3978', 'dislikeCount': '129', 'favoriteCount': '0', 'commentCount': '180'}}


In [72]:
# No. of comments for a video
responseVideo['items'][0]['snippet']['title']


'3 Chutneys Recipe | Meethi Chutney, Allam / Ginger Red Pachadi, Curry Leaf-Cashew Green Chutney'

In [59]:
int(21)//20

1